<a href="https://colab.research.google.com/github/imryoung9898/EconAI_Sentinel2/blob/main/1_sentinel_stack_and_reproject_IJ_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code does : <br>
1.   Stack three (R,G,B) bands
2.   Reproject and download


Last modified : Jan 14 2022 by Imryoung Jeong (neptune0118@gmail.com) <br>

Recommend to use Google Chrome to download zipfile automatically

# Set Up

In [ ]:
# pip install rioxarray
# pip install fiona
# pip install rasterio

### Import libraries

In [ ]:
import rioxarray as rxr
import rasterio as rio
import fiona, os, time, datetime
import requests, zipfile, shutil, re

from os import listdir
from tqdm import trange, notebook
from rasterio.crs import CRS
from google.colab import files
from urllib.parse import unquote 
from tqdm import tqdm

### Define directories & url

In [ ]:
url = 'https://www.dropbox.com/s/529mhfu07sp36u4/test_sentinel.zip?dl=0'
#drop_name = 'sentinel.zip'
drop_path = './Sentinel_image/'

# Functions

In [ ]:
def download_dropbox(url, folder):
  ## version 2.0, last modified by Hyunjoo Yang (hyang@sogang.ac.kr) on Jan. 14 2022
  ## This function downloads dropbox shareable link to a local folder (tested for file downloading, but not for shared folder!)
  
  # url: dropbox shareable link for downloading
  # folder: where to download
  
  headers = {'user-agent': 'Wget/1.16 (linux-gnu)'}
  r = requests.get(url, stream=True, headers=headers)
  
  # convert dropbox shareable link ("dl=0" to "dl=1")
  url = url.replace("?dl=0", "?dl=1")

  # check if the url returns valid status code (200)
  if r.status_code == 200:
    print('The url is valid.')

    # grab filename from the url, using regular expressions (and replace space to "_" )
    file_name = unquote(re.search(r'\/([^\/]+\.([\w]+))\?dl=([01])$', url).group(1)).replace(" ", "_")

    folder_n_fname = os.path.join(folder, file_name)

    # download
    print('Begin downloading < {} >'.format(folder_n_fname))


    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte
    t = tqdm(total=total_size, unit='B', unit_scale=True)
    with open(file_name, 'wb') as f:
      for data in r.iter_content(block_size):
        t.update(len(data))
        f.write(data)
    t.close()

    print('Finished downloading < {} >'.format(folder_n_fname))

    # unzip if zip file
    if file_name.endswith('.zip'):
      print('Extracting zip file...')
      with zipfile.ZipFile(file_name,"r") as zip_ref:
        zip_ref.extractall(folder)
    else:
      if not os.path.exists(folder):
        os.makedirs(folder)
      shutil.move(file_name, folder_n_fname)

    print('Done!')
  else:
    raise ValueError('Nothing to download: dropbox link is not valid. Check the URL link again')

In [ ]:
def stack_sentinel_bands(img_nm) :
    # version 1.0
    ## code to stack three RGB bands 
    ### img_nm: name of satellite imageries (e.g. 52SBC,2018-05-23,0)

    bands = [drop_path + "{}/R10m/B04.jp2".format(img_nm), #R
             drop_path + "{}/R10m/B03.jp2".format(img_nm), #G
             drop_path + "{}/R10m/B02.jp2".format(img_nm)] #B

    # Read metadata of first file
    with rio.open(bands[0]) as src0: meta = src0.meta

    # Update meta to reflect the number of layers
    meta.update(count = len(bands))

    # Read each layer and write it to stack
    with rio.open(drop_path + "{}.tif".format(img_nm), 'w', **meta) as dst:
        for id, layer in enumerate(bands, start=1):
            with rio.open(layer) as src1:
                dst.write_band(id, src1.read(1))


def sentinel_reprojector(img_nm) :
    # version 1.3, last modified by Imryoung Jeong (neptune0118@gmail.com) on Jan 14 2022
    ## This function reprojects raster to EPSG4326
    
    # Create a rasterio crs object for wgs 84 crs - lat / lon
    crs_wgs84 = CRS.from_string('EPSG:4326')
    
    original = rxr.open_rasterio(drop_path + "{}.tif".format(img_nm)).squeeze()
    original_reproj = original.rio.reproject(crs_wgs84)

    if not os.path.exists("./Sentinel_reproj/") : os.makedirs("./Sentinel_reproj/")
    
    out_dir = "./Sentinel_reproj/{}_EPSG4326.tif".format(img_nm)
    original_reproj.rio.to_raster(out_dir)

    # Download (very slow)
    files.download(out_dir)

# Loop over...

### Download and unzip Sentinel zipfile

In [ ]:
download_dropbox(url,drop_path)

The url is valid.
Begin downloading < ./Sentinel_image/test_sentinel.zip >


100%|██████████| 559M/559M [00:13<00:00, 41.6MB/s]


Finished downloading < ./Sentinel_image/test_sentinel.zip >
Extracting zip file...
Done!


In [ ]:
img_nm_list = [f for f in os.listdir(drop_path) if f.startswith("5")]
print(img_nm_list)

['52SDH,2018-06-24,0', '52SBC,2018-05-23,0']


In [ ]:
for img_nm in notebook.tqdm(img_nm_list) :

    stack_sentinel_bands(img_nm)

    sentinel_reprojector(img_nm)